<a href="https://colab.research.google.com/github/cyclingdata/Birds/blob/master/Birds_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computer Vision: Kookaburra or cockatoo?

This is an adaptation of the famous Kaggle competition cats vs dogs. I just decided to make this a little more 'Australian' :)

I have downloaded 500 pictures of Kookaburras and cockatoos, in two different directories


The first step is to connect to my Google account as I stored the pictures on a GCS bucket

In [0]:
# Authenticate.
from google.colab import auth
auth.authenticate_user()

Copy the two directories: one for Kookaburras, one for cockatoos

In [0]:
import os

try:
  os.mkdir('/tmp/images/')
  os.mkdir('/tmp/images/Cockatoos')
  os.mkdir('/tmp/images/Kookaburras')
except OSError:
    pass


In [3]:
!gsutil -m cp -r  gs://kooka-or-cockatoo/images/Cockatoos /tmp/images/
!gsutil -m cp -r  gs://kooka-or-cockatoo/images/Kookaburras /tmp/images/

Copying gs://kooka-or-cockatoo/images/Cockatoos/1300307_600_450_2_0_fit_0_ddca08a92d57c547ab27ee07057aceae.jpg...
Copying gs://kooka-or-cockatoo/images/Cockatoos/130222_web.jpg...
Copying gs://kooka-or-cockatoo/images/Cockatoos/146741142-jurong-bird-park-yellow-crested-cockatoo-singapore-observing.jpg...
Copying gs://kooka-or-cockatoo/images/Cockatoos/0a04f4bd53cd10218cf7b41623cf2f6e.jpg...
Copying gs://kooka-or-cockatoo/images/Cockatoos/01-0715_l_600x.progressive.jpg...
Copying gs://kooka-or-cockatoo/images/Cockatoos/10899962-3x2-940x627-e1554331787679.jpg...
Copying gs://kooka-or-cockatoo/images/Cockatoos/170px-Sulphur_Crested_Cockatoo_Nov10.jpg...
Copying gs://kooka-or-cockatoo/images/Cockatoos/117500151.jpg...
Copying gs://kooka-or-cockatoo/images/Cockatoos/117500131.jpg...
Copying gs://kooka-or-cockatoo/images/Cockatoos/18814515-cockatoo-in-botanic-garden-of-sydney-australia.jpg...
Copying gs://kooka-or-cockatoo/images/Cockatoos/2-yellow-crested-cockatoo-rob-downer.jpg...
Copying 

We need to split data into training and test. We will keep 70% of images for training and 30% for testing

In [4]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  # list files in source dir
  files = os.listdir(SOURCE)
  # split training validation
  nb_files = len(files)
  shuffle = random.sample(files,nb_files)
  training_files = shuffle[:round(SPLIT_SIZE*nb_files)]
  validation_files = shuffle[round(SPLIT_SIZE*nb_files):]
  for fname in training_files:
    if os.path.getsize(os.path.join(SOURCE,fname)) > 0:
      copyfile(os.path.join(SOURCE, fname),os.path.join(TRAINING, fname))
    else:
      print(fname + ' is zero length, so ignoring.')
  for fname in validation_files:
    if os.path.getsize(os.path.join(SOURCE,fname)) > 0:
      copyfile(os.path.join(SOURCE, fname),os.path.join(TESTING, fname))
    else:
      print(fname + ' is zero length, so ignoring.')

# create directories
try:
  os.mkdir("/tmp/kooka-v-cockatoo/")
  os.mkdir("/tmp/kooka-v-cockatoo/training/")
  os.mkdir("/tmp/kooka-v-cockatoo/training/Kookaburras")
  os.mkdir("/tmp/kooka-v-cockatoo/training/Cockatoos")
  os.mkdir("/tmp/kooka-v-cockatoo/testing/")
  os.mkdir("/tmp/kooka-v-cockatoo/testing/Kookaburras")
  os.mkdir("/tmp/kooka-v-cockatoo/testing/Cockatoos")
except OSError:
    pass

KOOKA_SOURCE_DIR = "/tmp/images/Kookaburras/"
TRAINING_KOOKA_DIR = "/tmp/kooka-v-cockatoo/training/Kookaburras/"
TESTING_KOOKA_DIR = "/tmp/kooka-v-cockatoo/testing/Kookaburras/"
COCKATOO_SOURCE_DIR = "/tmp/images/Cockatoos/"
TRAINING_COCKATOO_DIR = "/tmp/kooka-v-cockatoo/training/Cockatoos/"
TESTING_COCKATOO_DIR = "/tmp/kooka-v-cockatoo/testing/Cockatoos/"

split_size = .7
split_data(KOOKA_SOURCE_DIR, TRAINING_KOOKA_DIR, TESTING_KOOKA_DIR, split_size)
split_data(COCKATOO_SOURCE_DIR, TRAINING_COCKATOO_DIR, TESTING_COCKATOO_DIR, split_size)

In [5]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False
  
# pre_trained_model.summary()

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

--2020-03-26 02:49:17--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.203.128, 2404:6800:4008:c05::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M  43.0MB/s    in 2.0s    

2020-03-26 02:49:19 (43.0 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
last layer output shape:  (None, 7, 7, 768)


In [11]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "/tmp/kooka-v-cockatoo/training/"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = "/tmp/kooka-v-cockatoo/testing/"
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
	VALIDATION_DIR,
	target_size=(150,150),
	class_mode='binary'
)

from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 256 hidden units and ReLU activation
x = layers.Dense(128, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.5)(x)                  
# Add a final sigmoid layer for classification
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model( pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(lr=0.00001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])

Found 335 images belonging to 2 classes.
Found 143 images belonging to 2 classes.


In [12]:
history = model.fit_generator(
            train_generator,
            validation_data = validation_generator,
            steps_per_epoch = 30,
            epochs = 70,
            validation_steps = 10,
            verbose = 1)

Epoch 1/70
30/30 [==============================] - 18s 586ms/step - loss: 0.6703 - acc: 0.5831 - val_loss: 0.5473 - val_acc: 0.7273
Epoch 2/70
30/30 [==============================] - 16s 533ms/step - loss: 0.6225 - acc: 0.6876 - val_loss: 0.4162 - val_acc: 0.9091
Epoch 3/70
30/30 [==============================] - 16s 522ms/step - loss: 0.5782 - acc: 0.7462 - val_loss: 0.3415 - val_acc: 0.9091
Epoch 4/70
30/30 [==============================] - 13s 436ms/step - loss: 0.5415 - acc: 0.7712 - val_loss: 0.2663 - val_acc: 0.9510
Epoch 5/70
30/30 [==============================] - 16s 548ms/step - loss: 0.4775 - acc: 0.8460 - val_loss: 0.2321 - val_acc: 0.9510
Epoch 6/70
30/30 [==============================] - 17s 554ms/step - loss: 0.4357 - acc: 0.8693 - val_loss: 0.1651 - val_acc: 0.9580
Epoch 7/70
30/30 [==============================] - 15s 487ms/step - loss: 0.4025 - acc: 0.8735 - val_loss: 0.1452 - val_acc: 0.9510
Epoch 8/70
30/30 [==============================] - 14s 455ms/step - 